<a href="https://colab.research.google.com/github/adidror005/youtube-videos/blob/main/TradingStream2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

### Install Python SDK

In [1]:
!pip install alpaca-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 kB 2.8 MB/s eta 0:00:00


### Grab API key and Secret

In [2]:
from google.colab import userdata
ALPACA_API_KEY = userdata.get('ALPACA_API_KEY')
ALPACA_API_SECRET = userdata.get('ALPACA_API_SECRET')

### Connect to Trading Client

In [3]:
from alpaca.trading.client import TradingClient
trading_client = TradingClient(ALPACA_API_KEY, ALPACA_API_SECRET, paper=True)

### Helper Function to View Respones as DataFrames

In [4]:
import pandas as pd

class Util:
    @staticmethod
    def to_dataframe(data):
        if isinstance(data, list):
            return pd.DataFrame([item.__dict__ for item in data])
        return pd.DataFrame(data, columns=['tag', 'value']).set_index('tag')

### Cancel all Orders

In [5]:
trading_client.cancel_orders()

[]

# Make a Buy Order for 1000$ of AMD

In [13]:
from alpaca.trading.requests import MarketOrderRequest,TakeProfitRequest,LimitOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce, OrderClass

trading_client.submit_order(
    LimitOrderRequest(
    symbol='AMD',
    notional=1000,
    side=OrderSide.BUY,
    order_class = OrderClass.OTO,
    take_profit=TakeProfitRequest(limit_price=115.0),
    extended_hours=True,
    limit_price = 114.3,
    time_in_force=TimeInForce.DAY)
)


APIError: {"code":40310000,"message":"oto orders do not support extended hours trading"}

## Let's look at open orders

In [ ]:
from alpaca.trading.enums import QueryOrderStatus
from alpaca.trading.requests import GetOrdersRequest
Util.to_dataframe(trading_client.get_orders(GetOrdersRequest(status=QueryOrderStatus.OPEN)))

,id,client_order_id,created_at,updated_at,submitted_at,filled_at,expired_at,expires_at,canceled_at,failed_at,...,limit_price,stop_price,status,extended_hours,legs,trail_percent,trail_price,hwm,position_intent,ratio_qty
0,4954b422-9158-4974-8e8c-0b04dd54b29e,3bbe2ed9-fa9d-4629-acdb-db757d2950a2,2025-05-12 09:35:06.355465+00:00,2025-05-12 09:35:06.365818+00:00,2025-05-12 09:35:06.364437+00:00,None,None,2025-05-13 00:00:00+00:00,None,None,...,114.3,None,OrderStatus.NEW,True,None,None,None,None,PositionIntent.BUY_TO_OPEN,None


# StockTradingStream Example
* Similar for Crypto + Options

In [ ]:
from alpaca.data.live import StockDataStream
from alpaca.data.enums import DataFeed
stock_data_stream = StockDataStream(ALPACA_API_KEY, ALPACA_API_SECRET,feed=DataFeed.SIP)

### Nest_Asyncio: Makes Asyncio Work with Jupyer
* Credit Ewald De Wit

In [ ]:
import nest_asyncio
nest_asyncio.apply()

### Define Cooroutines

In [ ]:
async def on_quote(quote):
    print("New Quote")
    print(quote)

async def on_trade(trade):
    print("New Trade")
    print(trade)

async def on_1min_bar(bar):
    print("New 1 minute Bar")
    print(bar)

### Subscribe to Quotes, Trades and Bars

In [ ]:
stock_data_stream.subscribe_quotes(on_quote,"AMD")
stock_data_stream.subscribe_trades(on_trade,*["AMD","NVDA"])
stock_data_stream.subscribe_bars(on_1min_bar,"AMD",'NVDA','MU')

stock_data_stream.run()

New Trade
symbol='NVDA' timestamp=datetime.datetime(2025, 5, 12, 9, 35, 6, 975371, tzinfo=datetime.timezone.utc) exchange='P' price=122.58 size=4.0 id=15509 conditions=['@', 'T', 'I'] tape='C'
New Trade
symbol='NVDA' timestamp=datetime.datetime(2025, 5, 12, 9, 35, 6, 975373, tzinfo=datetime.timezone.utc) exchange='P' price=122.58 size=1.0 id=15510 conditions=['@', 'T', 'I'] tape='C'
New Trade
symbol='NVDA' timestamp=datetime.datetime(2025, 5, 12, 9, 35, 7, 517971, tzinfo=datetime.timezone.utc) exchange='K' price=122.58 size=100.0 id=16256 conditions=['@', 'T'] tape='C'
New Trade
symbol='AMD' timestamp=datetime.datetime(2025, 5, 12, 9, 35, 7, 584293, tzinfo=datetime.timezone.utc) exchange='K' price=110.04 size=60.0 id=2940 conditions=['@', 'T', 'I'] tape='C'
New Trade
symbol='AMD' timestamp=datetime.datetime(2025, 5, 12, 9, 35, 7, 584296, tzinfo=datetime.timezone.utc) exchange='K' price=110.04 size=40.0 id=2941 conditions=['@', 'T', 'I'] tape='C'
New Quote
symbol='AMD' timestamp=datetim

In [ ]:
await stock_data_stream.unsubscribe_bars("AMD")